In [15]:
import keras
from functools import reduce
import re
import numpy as np
import nltk
import json
from pprint import pprint as pp
from numpy import newaxis

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import LSTM
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [2]:
def tokenize(sent):
    """Returns the tokens of a sequece"""
    tokens = nltk.word_tokenize(sent)
    tokens = [w.lower() for w in tokens]
    return tokens

In [3]:
with open('data/SciQ dataset/train.json', 'r') as rf:
    train = json.load(rf)
with open('data/SciQ dataset/test.json', 'r') as rf:
    test = json.load(rf)
with open('data/SciQ dataset/valid.json', 'r') as rf:
    valid = json.load(rf)

In [79]:
import random
def preprocess(data_in):
    q = []
    s = []
    o = []
    l = []
    for sample in data_in:
        question = sample['question']
        support = sample['support']
        option1 = (sample['distractor1'], -1)
        option2 = (sample['distractor2'], -1)
        option3 = (sample['distractor3'], -1)
        option4 = (sample['correct_answer'], 1)
        options = [option1, option2, option3, option4]
        random.seed(1204)
        random.shuffle(options)
        q.append(question)
        s.append(support)
        o.append(tuple(op for op,_ in options))
        l.append(tuple(label for _l, label in options))
    X = {'questions': q, 'support': s, 'options': o}
    return X, l

In [59]:
def createVocab(input_data):
    vocab_list = set()
    for sample in input_data:
        s_t = tokenize(sample['support'])
        q_t = tokenize(sample['question'])
        d1_t = tokenize(sample['distractor1'])
        d2_t = tokenize(sample['distractor2'])
        d3_t = tokenize(sample['distractor3'])
        a_t = tokenize(sample['correct_answer'])
        vocab_list |= set(s_t+q_t+d1_t+d2_t+d3_t+a_t)
    vocab_list=sorted(vocab_list)
    vocab_size = len(vocab_list)+3
    vocab = dict((c,i+2) for i,c in enumerate(vocab_list))
    print("Vocab ready")
    return vocab_list, vocab_size, vocab

In [60]:
vocab_list, vocab_size, vocab = createVocab(train+valid+test)

Vocab ready


In [80]:
def get_vectors(input_sent, vocab, vocab_list):
    tokenized = tokenize(input_sent)
    vectorized = []
    for w in tokenized:
        if w in vocab_list:
            vectorized.append(vocab[w])
        else:
            vectorized.append(vocab['UNK_ID'])
    return vectorized

In [81]:
def vectorize_input(X, y, vocab, vocab_size, support_maxlen, query_maxlen):
    op1 = []
    op2 = []
    op3 = []
    op4 = []
    l1 = []; l2 = []; l3 = []; l4 = []
    for label_list in y:
        l1.append(label_list[0])
        l2.append(label_list[1])
        l3.append(label_list[2])
        l4.append(label_list[3])
    labels = [np.array(l1),np.array(l2),np.array(l3),np.array(l4)]
    qs = [get_vectors(sent, vocab, vocab_list) for sent in X['questions']]
    sps = [get_vectors(sent, vocab, vocab_list) for sent in X['support']]
    for sample_options in X['options']:
        op1.append(get_vectors(sample_options[0], vocab, vocab_list))
        op2.append(get_vectors(sample_options[1], vocab, vocab_list))
        op3.append(get_vectors(sample_options[2], vocab, vocab_list))
        op4.append(get_vectors(sample_options[3], vocab, vocab_list))
    return(pad_sequences(qs, maxlen=query_maxlen),\
           pad_sequences(sps, maxlen=support_maxlen),\
           pad_sequences(op1, maxlen=query_maxlen),\
           pad_sequences(op2, maxlen=query_maxlen),\
           pad_sequences(op3, maxlen=query_maxlen),\
           pad_sequences(op4, maxlen=query_maxlen),\
           labels
          )

In [82]:
EMBED_SIZE = 300
Q_HIDDEN_SIZE = 100
S_HIDDEN_SIZE = 300
BATCH_SIZE = 32
EPOCHS = 10
print('LSTM/EMBED/SUPPORT/QUERY={0},{1},{2},{3}'.format(LSTM,
                                                    EMBED_SIZE,
                                                    S_HIDDEN_SIZE,
                                                    Q_HIDDEN_SIZE))


LSTM/EMBED/SUPPORT/QUERY=<class 'keras.layers.recurrent.LSTM'>,300,300,60


In [83]:
support = layers.Input(shape=(S_HIDDEN_SIZE,), dtype='int32', name='support_input')
encoded_support = layers.Embedding(vocab_size, EMBED_SIZE)(support)
encoded_support = layers.Dropout(0.3)(encoded_support)
support_LSTM = LSTM(EMBED_SIZE)(encoded_support)
support_LSTM = layers.RepeatVector(S_HIDDEN_SIZE)(support_LSTM)

In [84]:
question = layers.Input(shape=(Q_HIDDEN_SIZE,), dtype='int32', name='question_input')
encoded_question = layers.Embedding(vocab_size, EMBED_SIZE)(question)
encoded_question = layers.Dropout(0.3)(encoded_question)
question_LSTM = LSTM(EMBED_SIZE)(encoded_question)
question_LSTM = layers.RepeatVector(S_HIDDEN_SIZE)(question_LSTM)

In [85]:
MatchLSTM_layer = layers.add([support_LSTM, question_LSTM])
MatchLSTM_layer = layers.Dropout(0.5)(MatchLSTM_layer)
MatchLSTM_layer = LSTM(EMBED_SIZE, return_sequences=True)(MatchLSTM_layer)
MatchLSTM_layer = layers.Dense(S_HIDDEN_SIZE, activation = 'softmax')(MatchLSTM_layer)
print(MatchLSTM_layer)

Tensor("dense_3/truediv:0", shape=(?, 300, 300), dtype=float32)


In [86]:
def get_option_match(distractor, MatchLSTM_layer, option_num, S_HIDDEN_SIZE, Q_HIDDEN_SIZE):
    encoded_distractor = layers.Embedding(vocab_size, EMBED_SIZE)(distractor)
    encoded_distractor = layers.Dropout(0.3)(encoded_distractor)
    distractor_LSTM = LSTM(EMBED_SIZE)(encoded_distractor)
    added = layers.add([MatchLSTM_layer, distractor_LSTM])
    option_Match = layers.Dropout(0.3)(added)
    option_Match = LSTM(1)(option_Match)
    option_Match = layers.Dense(1, activation='tanh', name='op_{0}'.format(str(option_num)))(option_Match)
    return option_Match

In [87]:
distractor1 = layers.Input(shape=(Q_HIDDEN_SIZE,), dtype='int32', name='Option1_Input')
distractor2 = layers.Input(shape=(Q_HIDDEN_SIZE,), dtype='int32', name='Option2_Input')
distractor3 = layers.Input(shape=(Q_HIDDEN_SIZE,), dtype='int32', name='Option3_Input')
distractor4 = layers.Input(shape=(Q_HIDDEN_SIZE,), dtype='int32', name='Option4_Input')

In [88]:
option1_Match = get_option_match(distractor1,MatchLSTM_layer,1, S_HIDDEN_SIZE,Q_HIDDEN_SIZE)
option2_Match = get_option_match(distractor2,MatchLSTM_layer,2, S_HIDDEN_SIZE,Q_HIDDEN_SIZE)
option3_Match = get_option_match(distractor3,MatchLSTM_layer,3, S_HIDDEN_SIZE,Q_HIDDEN_SIZE)
option4_Match = get_option_match(distractor4,MatchLSTM_layer,4, S_HIDDEN_SIZE,Q_HIDDEN_SIZE)

In [89]:
model = Model([support, question, distractor1, distractor2, distractor3, distractor4], \
              [option1_Match, option2_Match, option3_Match, option4_Match])
print(model.summary())
plot_model(model, to_file='Final_Model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
support_input (InputLayer)      (None, 300)          0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 300, 300)     9234600     support_input[0][0]              
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, 60, 300)      9234600     question_input[0][0]             
__________________________________________________________________________________________________
dropout_33

In [90]:
model.compile(optimizer='adam',
             loss='squared_hinge',
             metrics=['accuracy'])

In [93]:
early_stop = EarlyStopping(monitor=['op_1_loss','op_2_loss','op_3_loss','op_4_loss'],
                              min_delta=0,
                              patience=10,
                              verbose=1, mode='auto')

In [94]:
print('Training')
X,y = preprocess(train)
q,s,d1,d2,d3, d4,a = vectorize_input(X, y, vocab, vocab_size, S_HIDDEN_SIZE, Q_HIDDEN_SIZE)

Training


In [95]:
# checkpoint
filepath="\models\weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='op_1_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [96]:
model.fit([s,q, d1,d2,d3,d4], [a[0],a[1], a[2], a[3]],
         batch_size=128,
         epochs=50)

Epoch 1/50
11679/11679 [==============================] - 279s 24ms/step - loss: 1.9889 - op_1_loss: 0.8009 - op_2_loss: 0.0543 - op_3_loss: 0.4176 - op_4_loss: 0.7161 - op_1_acc: 0.0000e+00 - op_2_acc: 0.9447 - op_3_acc: 0.0000e+00 - op_4_acc: 0.0000e+00
Epoch 2/50
11679/11679 [==============================] - 268s 23ms/step - loss: 1.2647 - op_1_loss: 0.5326 - op_2_loss: 0.0162 - op_3_loss: 0.2491 - op_4_loss: 0.4667 - op_1_acc: 0.0000e+00 - op_2_acc: 1.0000 - op_3_acc: 0.5287 - op_4_acc: 0.0000e+00
Epoch 3/50
11679/11679 [==============================] - 268s 23ms/step - loss: 0.8570 - op_1_loss: 0.3624 - op_2_loss: 0.0121 - op_3_loss: 0.1666 - op_4_loss: 0.3160 - op_1_acc: 0.0000e+00 - op_2_acc: 1.0000 - op_3_acc: 1.0000 - op_4_acc: 0.0000e+00
Epoch 4/50
11679/11679 [==============================] - 267s 23ms/step - loss: 0.6040 - op_1_loss: 0.2550 - op_2_loss: 0.0093 - op_3_loss: 0.1174 - op_4_loss: 0.2224 - op_1_acc: 0.4630 - op_2_acc: 1.0000 - op_3_acc: 1.0000 - op_4_acc: 0.8

11679/11679 [==============================] - 266s 23ms/step - loss: 0.0127 - op_1_loss: 0.0051 - op_2_loss: 3.3300e-04 - op_3_loss: 0.0027 - op_4_loss: 0.0046 - op_1_acc: 1.0000 - op_2_acc: 1.0000 - op_3_acc: 1.0000 - op_4_acc: 1.0000
Epoch 35/50
11679/11679 [==============================] - 267s 23ms/step - loss: 0.0119 - op_1_loss: 0.0047 - op_2_loss: 3.1244e-04 - op_3_loss: 0.0026 - op_4_loss: 0.0043 - op_1_acc: 1.0000 - op_2_acc: 1.0000 - op_3_acc: 1.0000 - op_4_acc: 1.0000
Epoch 36/50
11679/11679 [==============================] - 267s 23ms/step - loss: 0.0111 - op_1_loss: 0.0044 - op_2_loss: 2.9343e-04 - op_3_loss: 0.0024 - op_4_loss: 0.0040 - op_1_acc: 1.0000 - op_2_acc: 1.0000 - op_3_acc: 1.0000 - op_4_acc: 1.0000
Epoch 37/50
11679/11679 [==============================] - 266s 23ms/step - loss: 0.0104 - op_1_loss: 0.0041 - op_2_loss: 2.7582e-04 - op_3_loss: 0.0023 - op_4_loss: 0.0037 - op_1_acc: 1.0000 - op_2_acc: 1.0000 - op_3_acc: 1.0000 - op_4_acc: 1.0000
Epoch 38/50
1167

In [97]:
X_valid, y_valid = preprocess(valid)
vq,vs,vd1,vd2,vd3,vd4,va = vectorize_input(X_valid, y_valid, vocab, vocab_size, S_HIDDEN_SIZE, Q_HIDDEN_SIZE)
loss = model.evaluate([vs,vq, vd1,vd2,vd3,vd4], [va[0],va[1], va[2], va[3]],
                         batch_size=BATCH_SIZE)
# print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

10/10 [==============================] - 3s 331ms/step


In [98]:
p_op1,p_op2, p_op3, p_op4 = model.predict([vs,vq, vd1,vd2,vd3,vd4])

In [99]:
model.save_weights("model_50_epochs_bs_128.h5")
from keras.models import model_from_json
model_json = model.to_json()
with open("model_50_epochs_bs_128.json", "w") as wf:
    wf.write(model_json)

In [100]:
# y_valid[:10]

In [101]:
# print(y[:32])

In [102]:
predictions = [[o1,o2,o3,o4] for o1,o2,o3,o4 in zip(p_op1,p_op2,p_op3,p_op4)]

In [103]:
from Visualize import *

In [104]:
print(highest_val_correct_acc(predictions,y_valid))

100.0
